# Kaggle Competition | Titanic Machine Learning from Disaster

> The sinking of the RMS Titanic is one of the most infamous shipwrecks in history. On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.
>
> One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.
>
> In this contest, we ask you to complete the analysis of what sorts of people were likely to survive. In particular, we ask you to apply the tools of machine learning to predict which passengers survived the tragedy.
>
> This Kaggle Getting Started Competition provides an ideal starting place for people who may not have a lot of experience in data science and machine learning."

From the competition [homepage](https://www.kaggle.com/c/titanic).

## This notebook

The aim of this notebook is to give you some insight into a more "real" world problem where are data isn't cleaned for us. We need to explore the data, clean it and then apply some ML methods.

First we'll need to install some dependencies and then import them.

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from patsy import dmatrices
from sklearn import datasets, svm
from statsmodels.nonparametric import smoothers_lowess
from statsmodels.nonparametric.kde import KDEUnivariate

## The data

First let's read in a data and see what we're dealing with.

In [31]:
df = pd.read_csv("data/train.csv") 
print(fr"{df.shape[0]} rows, {df.shape[1]} columns")
df.head()

891 rows, 12 columns


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Missing values

One thing we want to watch out for is how complete our dataset is. First, let's have a look at the number of rows and how many entries are non-null.

In [25]:
percentage_complete = df.notnull().sum() / df.shape[0]
percentage_complete.map("{:.1%}".format)

PassengerId    100.0%
Survived       100.0%
Pclass         100.0%
Name           100.0%
Sex            100.0%
Age             80.1%
SibSp          100.0%
Parch          100.0%
Ticket         100.0%
Fare           100.0%
Cabin           22.9%
Embarked        99.8%
dtype: object

Clearly we can see that _Cabin_ in particular has a significant number of entries missing. How can we deal with that? At this point it's probably worth just removing it from our dataset.

In [26]:
df.drop(["Cabin"], axis=1, inplace=True)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


Now that we're removed _Cabin_ we're safe to drop all remaining NaN values. If we'd ran this line first our dataset would have been ~23% of it's original size!

In [28]:
df.dropna(inplace=True)
df.shape

(712, 11)

## Exploring the data visually